In [1]:
#Load Library
import pandas as pd
from sklearn.decomposition import NMF
#import pyarrow
#import pyarrow.feather as feather
import feather

### 1. Import Data 

In [2]:

df_original = pd.read_csv('final/MOD-00067.final.csv')
df_original.shape

(165865, 55)

In [3]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165865 entries, 0 to 165864
Data columns (total 55 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165865 non-null  int64  
 1   timestamp        165865 non-null  object 
 2   timestamp_local  165865 non-null  object 
 3   sn               165865 non-null  object 
 4   rh               165865 non-null  float64
 5   temp             165865 non-null  float64
 6   bin0             165865 non-null  float64
 7   bin1             165865 non-null  float64
 8   bin2             165865 non-null  float64
 9   bin3             165865 non-null  float64
 10  bin4             165865 non-null  float64
 11  bin5             165865 non-null  float64
 12  bin6             165865 non-null  float64
 13  bin7             165865 non-null  float64
 14  bin8             165865 non-null  float64
 15  bin9             165865 non-null  float64
 16  bin10            165865 non-null  floa

### 2. Set up which column to include for NMF (ARISense gas data and MODULAIR-PM aerosol data)

In [4]:
N_COMPONENTS = 4

#Columns to include in the analysis is bin 0 to bin 23 as well as gases
COLS_TO_INCLUDE = []
for i in range(3):
    COLS_TO_INCLUDE.append('bin'+str(i))
COLS_TO_INCLUDE.append('co')
COLS_TO_INCLUDE.append('no')
COLS_TO_INCLUDE.append('no2')
COLS_TO_INCLUDE.append('o3')

In [5]:
COLS_TO_INCLUDE 

['bin0', 'bin1', 'bin2', 'co', 'no', 'no2', 'o3']

In [6]:
# For the df_original, have the same number of rows as df that will become an input for NMF
# So in the future when we merge, they have the same rows of interest 
df_original = df_original.dropna(subset = COLS_TO_INCLUDE)
df_original = df_original.reset_index(drop=True)
df_original.shape

(12301, 73)

In [7]:
df = df_original.reset_index().drop('index',axis=1)
#Drop the rows that include NaN values for columns of interest 
df = df.dropna(subset=COLS_TO_INCLUDE)
df = df[COLS_TO_INCLUDE]
df

,bin0,bin1,bin2,co,no,no2,o3
0,12.914967,1.151983,0.323933,67.950000,3.750000,0.933333,29.316667
1,12.583917,1.092117,0.309133,63.916667,3.766667,0.950000,29.283333
2,12.070900,1.110283,0.294350,65.016667,3.983333,0.916667,29.083333
3,11.691667,1.097817,0.309200,66.540000,3.940000,0.700000,29.140000
4,11.416633,1.047700,0.283717,59.042857,3.700000,0.628571,29.042857
...,...,...,...,...,...,...,...
12296,15.742250,1.868767,0.422067,34.483333,6.750000,0.400000,28.900000
12297,15.923467,1.900750,0.395217,36.050000,5.633333,0.155556,29.700000
12298,16.075750,1.932817,0.416300,36.800000,5.750000,0.094444,29.516667
12299,16.161667,1.912083,0.410233,36.333333,6.083333,0.100000,29.183333


### 3. Set up the NMF
#### 3.1. Fit the data 
Learn a NMF model for the data X and returns the transformed data
* **X: Original Timeseries matrix**
    - 15 x 11,001 (15 features x 11,001 timeseries datapoints) 
* **W: Coefficient matrix** 
    - summation of the basis vector
    - 15 x 3 (15 features x 3 factors)
    - Which feature is particularly prominant? 
* **H: Basis matrix** 
    - Coefficient of the summation basis vector 
    - 3 x 11,001 (3 factors x 11,001 timeseries datapoints)
    - Corresponding coefficients for each timeseries data of each factor

Ex) 
* 1st row of W: [2.49, 2.92, 37.4] 
* 1st column of H: [0.75, 0.565, 0.324] 
* W x H = (2.49 * 0.75) + (2.92 * 0.565) + (37.4 * 0.324) = 15.634
* 1st X value of 1st timeseries data point at bin 0 is 15.95

In [8]:
#set up the NMF
nmf = NMF(n_components=N_COMPONENTS, alpha=0.1,max_iter=15000)

#fit the data (Note: input matrix has to be transposed)

#W = nmf.fit_transform(X=df_normalized_both[COLS_TO_INCLUDE].T)
W = nmf.fit_transform(X=df[COLS_TO_INCLUDE].T)
H = nmf.components_

#### 3.2. results matrix is trasposed coefficient matrix 
* Matrix size: 11,001 x 3

In [9]:
#Convert the results to a dataframe
results = pd.DataFrame(H.T,index=df.index)
#Set the column names
results.columns = ["Factor {}".format(i+1) for i in range(H.T.shape[1])]
results

,Factor 1,Factor 2,Factor 3,Factor 4
0,0.741517,0.448148,0.238807,0.101657
1,0.697169,0.447646,0.233507,0.102279
2,0.708824,0.444372,0.220595,0.110260
3,0.725588,0.445224,0.210656,0.108644
4,0.643638,0.443948,0.209489,0.100113
...,...,...,...,...
12296,0.367929,0.439339,0.326137,0.210188
12297,0.387235,0.452506,0.330778,0.169103
12298,0.395229,0.449572,0.333897,0.173531
12299,0.389454,0.444176,0.335740,0.185943


### 4. Calculate the Composition
#### 4.1. 3 factors and weights - how much feature is present in each factor
* Size: 3 x 15 

In [10]:
comp = pd.DataFrame(W.T,index=results.columns,columns=COLS_TO_INCLUDE)
comp

,bin0,bin1,bin2,co,no,no2,o3
Factor 1,2.621614,0.000000,0.000000,90.952624,0.000000,2.131462,0.000000
Factor 2,1.699793,0.000000,0.000000,0.000000,2.104884,0.848038,65.059216
Factor 3,40.985167,8.355595,2.423312,0.000000,0.000000,0.483607,0.000000
Factor 4,2.170366,0.000000,0.000000,4.774109,27.802872,0.000000,1.470990


### 5. Calculate Residual 
* Each column of res df = (total sum of W * H of feature of 1 factor)/(total sum of feature in orginal timeseries)
    - If value > 1: Overestimation
    - If value < 1: underestimation
* Residual column = 1 - (summation of W * H of 1 feature of 3 factors) 
    - If Residual < 0: overestimation
    - If Residual > 0: Underestimation

In [11]:
#calculate the total and residual for each column

res = []
# [[0, bin0], [1, bin1],...[14,o3]] 
for i, col in enumerate(comp.columns):
    #extract each bin's column for the weights of each factor and 
    #multiply that weights to the basis matrix 
    #Then sum up each factor's weight*basis matrix values
    by_factor = pd.DataFrame(comp.iloc[:, i].values * H.T).sum()
        
   # divide by the total ammount for a given species/column
    by_factor /= df[col].sum()
    #by_factor /= df_normalized_both[col].sum()
    
    #Transpose the by_factor and add that as a row to the "res" dataframe
    res.append(pd.DataFrame(by_factor, columns=[col]).T)

#Converting to dataframe
res = pd.concat(res)

#Columns of res df is the same as results (coefficient matrix) columns
res.columns = results.columns

#add a residual column which simply adds up across the row and subtract that from 1
res['Residual'] = 1-res.sum(axis=1)

#If res > 0, underestimate, if res < 0, overestimate
res

,Factor 1,Factor 2,Factor 3,Factor 4,Residual
bin0,0.144083,0.059048,0.766969,0.029223,0.000677
bin1,0.000000,0.000000,1.124589,0.000000,-0.124589
bin2,0.000000,0.000000,1.157414,0.000000,-0.157414
co,0.987389,0.000000,0.000000,0.012697,-0.000087
no,0.000000,0.163416,0.000000,0.836625,-0.000041
no2,0.739073,0.185863,0.057097,0.000000,0.017967
o3,0.000000,0.991369,0.000000,0.008688,-0.000057


### 6. Merge the Results with the Original Time Series Imported
#### 6.1. Merge the Basis Matrix

In [12]:
results

,Factor 1,Factor 2,Factor 3,Factor 4
0,0.741517,0.448148,0.238807,0.101657
1,0.697169,0.447646,0.233507,0.102279
2,0.708824,0.444372,0.220595,0.110260
3,0.725588,0.445224,0.210656,0.108644
4,0.643638,0.443948,0.209489,0.100113
...,...,...,...,...
12296,0.367929,0.439339,0.326137,0.210188
12297,0.387235,0.452506,0.330778,0.169103
12298,0.395229,0.449572,0.333897,0.173531
12299,0.389454,0.444176,0.335740,0.185943


In [13]:
#Merge the basis matrix with the original timeseries
results = pd.merge(df_original, results,left_index=True,right_index=True,how='outer')

In [14]:
results.shape

(12301, 77)

### 7. Export Data
#### 7.1. Export the Merge Timeseries (Basis matrix) and Composition Result (Weight Matrix)

In [15]:
#export the timeseries data
folder = '/Users/laurayang/Dropbox/Dr.Ng Group/Yang/Project2-LCS/B_Data-ES&T Roof/munged/NMF Raw-ARISense-Data/interpolated-raw-gas/normalized-up-to-bin2/'
feather.write_feather(results.reset_index(),folder+'5a-nmf-tseries-results-6min-4-factors.feather')
results.to_csv(folder+'5a-nmf-tseries-results-6min-4-factors.csv')

#export the composition result 
feather.write_feather(res.reset_index(), folder+'5a-nmf-composition-results-6min-4-factors.feather')
res.to_csv(folder+'5a-nmf-composition-results-6min-4-factors.csv')